# S3 버전 관리 및 객체 잠금

## S3 버전 관리
- 객체의 생성, 업데이트, 삭제의 모든 단계를 저장
    - 삭제 시에는 실제 객체를 삭제하는 대신 삭제 마커를 추가 : 모든 버전을 제거해야지만 실제로 객체가 삭제됨
- 버킷 단위로 활성화 필요(비활성화가 디폴트) : 버전 관리를 활성화 하면, 같은 파일이 계속해서 늘어나게 되니까 저장 공간을 비효율적으로 쓰게 됨.
    - 중지 가능, 비활성화는 불가능
    - 한번 버전 관리를 시작하면 비활성화 불가능 (버킷 삭제 후 재생성으로 해결 가능)
- 수명 주기 관리와 연동 가능
- MFA 인증 후 삭제 기능을 통해 보안 강화 가능

## 버전 관리가 작동하는 흐름
1. 파일 생성
    - Key: test.jpg // id: 11111
2. 파일 업데이트
    - Key: test.jpg // id: 22222
3. 파일 삭제
    - Delete Marker 추가 (유저는 Delete라고 인식)
4. 파일 삭제 취소
    - Delete Marker 제거
    - 유저는 Key: test.jpg // id: 22222 로 인식
5. 22222 버전 삭제 (파일이 실제로 삭제)
    - Roleback : 11111 버전이 최신으로 바뀜
    - 결국 해당 파일을 실제로 삭제하려면, 모든 버전을 직접 삭제해야 함
    - 실제로 삭제할 때, MFA 인증을 요구하도록 만들어 보안을 강화할 수 있음

## S3 버전 관리 유의사항
- 모든 버전에 대한 비용 발생
    - 10GB 객체의 버전이 5개 있다면, 총 50GB에 대해 비용 발생

## S3 객체 잠금
- Write Once, Read Many(WORM) 모델을 활용하여 객체를 저장
- 고정된 시간, 혹은 무기한으로 객체의 삭제, 덮어쓰기 방지 기능
- 규정 준수 및 객체의 보호를 위해 사용
- 버전 관리 활성화 필요

## S3 객체 잠금의 종류
- 종류에 따라 두 가지 모드
    - 보관 모드(Retention Mode) : 일정 기간동안 수정 방지
        - 규정 준수 모드 : 누구도 해당 기간동안 잠금 설정 변경, 객체 삭제 불가능
        - 거버넌스 모드 : 특별한 권한 없이 삭제, 잠금 설정 변경 불가능
            - 객체 삭제 방지, 규정에 따라 보관하기 위해 사용
            - 규정 준수 모드를 사용하기 전 테스트 단계에서 사용
    - 법적 보족(Legel Hold)
        - Hold를 객체에 부여하고, Hold가 존재하는 한 객체 삭제, 수정 불가능
        - Hold를 관리할 수 있는 사용자를 생성하여 적용하는 방식
        - 제한 기간이 없음

### Demo - S3 버전 관리 테스트
1. S3 버킷 생성
    - 버킷 버전 관리 활성화
2. 파일 업로드
    - text.txt 업로드, 버전 한번 확인해보자
3. 파일 업데이트
    - text.txt 내용 바꿔서 업로드, 버전 확인해보자
4. 파일 삭제
    - 버킷에 아무것도 안보임
    - 버전 표시 버튼 누르면 각 버전과 삭제 마커가 보임
    - 삭제 마커를 지우면? 다시 파일이 보인다
5. 두번째 버전을 지워보자
    - 영구 삭제 후 다운로드 해보면 첫번째 버전
6. 진짜로 지우려면 모든 버전을 선택하고 삭제(영구 삭제)
7. 리소스 정리 - 버킷 삭제